## After you have analyzed all of the data and now you want to find out what the dynamics are within the samples

again, load in necessary python modules

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt

from IPython.html.widgets import interact, fixed
import ipywidgets as widgets

import io 

font_plt = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 10,
            }
font_plt_ax = {'family': 'serif',
               'color':  'black',
               'weight': 'normal',
               'size': 10,
              }
import sys
import glob #glob is helpful for searching for filenames or directories

In [ ]:
sys.path.append('Z:\\Code\\PythonCode\\Differential-Dynamic-Microscopy---Python') 
import ddm_clean as ddm #this is the module containing the differential dynamic microscopy code

In [ ]:
import scipy #scientific python
import pickle #for saving data

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import mpld3

# Use seaborn styles for nice-looking plots
#import seaborn; seaborn.set()

In [ ]:
newt = lambda t,s: (1./s)*gamma(1./s)*t
framerate = 18.0
px = 0.194
imDimension = 256
q = np.arange(0,imDimension/2)*2*np.pi*(1./(imDimension*(px))) 

Load in the data from step 2.

In [ ]:
datadirectory = "Z:\\Where\\Your\\Data\\theRAVs\\are\\stored\\"
datafile = "file_where_you_saved_data_from_step_2.p"
Data = pickle.load(open(datadirectory+datafile,'rb'))

In [ ]:
Topology = "linear"
Condition = "buffer"
video = "1"
roi = "0"

In [ ]:
ravs = Data['ravs_'+Topology+'_'+Condition+'_'+video+'_'+roi]
fits = Data['fitparams_'+Topology+'_'+Condition+'_'+video+'_'+roi]

After loading in all of the data, initialize the arrays that you will store the transport coefficients and alpha values within.

In [ ]:
alphs = np.zeros(numberofROIsandvideos)
difs = np.zeros(numberofROIsandvideos)

now we want to look at the newt taus (this "fixed" by the alpha value)

In [ ]:
tau0 = newt(fits[:,1],fits[:,3])

In [ ]:
l = 0 # change to represent the video number and ROI you are on. 

next you will choose a region (a range of q values) that you think are good.

In [ ]:
w = np.where((q<2.9)&(q>.88))
r = np.polyfit(np.log(q[w[0]]),np.log(tau0[w[0]]),1)

The next cell will plot the calculated transport and alpha value for the region you set above, if it doesn't fit weel enough keep changing the region. 

In [ ]:
power = r[0]
yint = np.exp((-1)*r[1])
alpha = 2.0/float((-1)*power)

fig = plt.figure();
ax = plt.gca()
ax.loglog(q[:-1],tau0,color='r',marker='o',linestyle ='',markersize=3,alpha=1)
ax.plot(q, (1./diffusion_coeff) * (q**power), '-k', label="K = %.3f , alpha = %.2f " % (diffusion_coeff, alpha))
ax.set_xlim(0.1,5.7)           
ax.set_xlabel('Spatial Frequencies', fontdict=font_plt_ax)
ax.set_ylabel('Decay Times', fontdict=font_plt_ax)
ax.legend()

once you have decided that the fits look good, you should save the important values into their arrays (which you do in the following cell)

In [ ]:
alphs[l] = alpha
diffusion_coeff = yint 
difs[l] = diffusion_coeff

Continue until you have looked at all of the videos you are trying to analyze (should be the same number as l)

You will then save the alphs and difs arrays as a dictionary and as a csv file (so that you have them saved in two places!!)

In [1]:
difs_alphs_dictionary = {} #comment out...

In [ ]:
difs_alphs_dictionary[Topology+" "+Condition+" "+video+" "+roi+" tansport coefficients"] = difs
difs_alphs_dictionary[Topology+" "+Condition+" "+video+" "+roi+" alpha values"] = alphs

In [ ]:
pickle.dump(difs_alphs_dictionary,open("whatever and whereever you want the file name to be .p",'wb'))
np.savetxt("YourDirectory/Difs_"+Topology+"_"+Condition+"_"+video+"_"+roi+".csv",difs,delimiter=',')
np.savetxt("YourDirectory/Alphas_"+Topology+"_"+Condition+"_"+video+"_"+roi+".csv",alphs,delimiter=',')